In [ ]:
pip install pmdarima

In [ ]:
import numpy as np
import pandas as pd
import os
import joblib
import matplotlib as mpl
import pmdarima as pm
import random
#import sktime
import sklearn
from pmdarima import auto_arima
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
from tqdm import tqdm
import pkg_resources

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else:
            #df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


'''
def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df
'''

def import_data(file_path):
    """create a dataframe from the kaggle input and optimize its memory usage"""
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            if filename == file_path:
                file = os.path.join(dirname, filename)
                df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
                df = reduce_mem_usage(df)
                return df
    raise FileNotFoundError("File not found in Kaggle input directory.")
    
    
def mean_absolute_scaled_error(y_true, y_pred, y_train):
    """
    Calculate the Mean Absolute Scaled Error (MASE) of the predictions.

    Args:
    y_true (array-like): The true values of the target variable.
    y_pred (array-like): The predicted values of the target variable.
    y_train (array-like): The true values of the target variable for the training set.

    Returns:
    float: The MASE of the predictions.
    """
    mae = mean_absolute_error(y_true, y_pred)
    mae_naive = np.mean(np.abs(y_train[1:].to_numpy() - y_train[:-1].to_numpy()))
    mase = mae / (mae_naive)
    return mase



# Load the data from the CSV file into a Pandas dataframe
total = import_data("total.csv")

# Check the memory usage of the optimized dataframe
print(total.info(memory_usage='deep'))

#total = total[(total['store'] == 548) | (total['store'] == 6100)]
total = total[total['store'] == 548]
total

In [ ]:
# Convert 'date' column to datetime format
total['date'] = pd.to_datetime(total['date'], format='%Y-%m-%d')

total['unique_id'] = total['item_nbr'].astype(str) + '_' + total['store'].astype(str)
total.drop(columns=['item_nbr', 'store'], inplace=True)
total.reset_index(drop=True, inplace=True)
total.drop(columns=['index'], inplace=True)
total = total[['date', 'unique_id'] + [col for col in total.columns if col not in ['date', 'unique_id']]]

#column with lagged values
total['unit_sales_lag7'] = total.groupby('unique_id')['unit_sales'].shift(7)
total.fillna(0, inplace=True)  # fill missing values with 

Optimization_Model = {}

# get unique series IDs
series_ids = total['unique_id'].unique()

###############################
#series_ids = total['unique_id'].unique()

# Select 10 random series IDs
#random_series_ids = random.sample(list(series_ids), 10)
#print(f"Selected random series IDs: {random_series_ids}")

# Replace series_ids with the 10 random_series_ids
#series_ids = random_series_ids
###############################

def fit_sarimax(series_id):
    
    # select data for the current series
    series = total[total['unique_id'] == series_id]
    series.set_index('date', inplace=True)
    
    # Convert data types to float32 or float64
    #series['unit_sales'] = series['unit_sales'].astype('float32')
    series.loc[:, 'unit_sales'] = series['unit_sales'].astype('float32')

    
    # Prepare the exogenous variables (the additional columns)
    exog = series.drop(columns=['unique_id', 'unit_sales']).astype('float32')
    
    # split the data into training and testing sets
    train_size = int(len(series) * 0.8)
    train, test = series[:train_size], series[train_size:]
    
    train_exog, test_exog = exog[:train_size], exog[train_size:]

    # replace NaN values with the mean of the series
    train = train.fillna(train.mean())

    # replace infinite values with the max/min value of the series
    train = train.replace([np.inf, -np.inf], [train.max(), train.min()])
    
    #print("Fitting SARIMAX model for series", series_id)

    # fit the SARIMAX model
    #model_sarimax = auto_arima(train['unit_sales'], X=train_exog, seasonal=True, m=7, stepwise=True)
    model_sarimax = auto_arima(train['unit_sales'], X=train_exog, seasonal=True, m=7, stepwise=True, trend='n')


    # get model forecasts
    sarimax_forecast, conf_int = model_sarimax.predict(n_periods=len(test), X=test_exog, return_conf_int=True)

    # calculate evaluation metrics
    sarimax_rmse = np.sqrt(mean_squared_error(test['unit_sales'], sarimax_forecast))
    sarimax_mae = mean_absolute_error(test['unit_sales'], sarimax_forecast)
    sarimax_r2 = r2_score(test['unit_sales'], sarimax_forecast)
    sarimax_mase = mean_absolute_scaled_error(test['unit_sales'], sarimax_forecast, train['unit_sales'])
    
    # update the Optimization_Model dictionary
    Optimization_Model[series_id] = {'order': model_sarimax.order,
                                     'seasonal order': model_sarimax.seasonal_order,
                                     'mae': sarimax_mae,
                                     'rmse': sarimax_rmse,
                                     'mase': sarimax_mase,
                                     'r2': sarimax_r2}

    #print("Fitted SARIMAX model for series", series_id)
    #print(Optimization_Model[series_id])
    #print("")

    # return the evaluation metrics
    return sarimax_rmse, sarimax_mae, sarimax_mase, sarimax_r2

# fit SARIMAX models for all series in parallel
results = Parallel(n_jobs=-1, verbose=10)(delayed(fit_sarimax)(series_id) for series_id in tqdm(series_ids, desc='Fitting SARIMAX models'))

# Create a dataframe of the results
results_df = pd.DataFrame(results, columns=['RMSE_SARIMAX', 'MAE_SARIMAX', 'MASE_SARIMAX', 'R2_SARIMAX'])
results_df.index = series_ids

# Convert the Optimization_Model dictionary to a DataFrame
model_params_df = pd.DataFrame.from_dict(Optimization_Model, orient='index')

# Join the results_df DataFrame with the model_params_df DataFrame
final_results_df = results_df.join(model_params_df)

# Save the final_results_df to a CSV file
final_results_df.to_csv('sarimax_results_and_params.csv', index=True)

In [ ]:
results_df